# 06.3 - Common Operations I: Join/Merge

In this and the next notebook we'll take a look at some common dataframe operations like __joins__ (aka __merges__), __group_by__, and how to sequence them.

Imagine handling an excel file. But the excel file is larger than the memory of your laptop. If you conceptualize a dask dataframe as a large excel sheet that has to be handled with special code, you will be able to conceptualize the transformations you wish to perform. If you have an SQL background, you will probably already know some of these operations.

---
## Join: Merge in Pandas/Dask

A Join is an operation in [relational databases](https://en.wikipedia.org/wiki/Relational_database) where you combine the registries between two tables. If you have an SQL background you should be familiar with this operation. With a join, you can match where two tables match under any combination of column values. The two tables are usually defined as __left__ and __right__, as these operations are normally non-commutative. The __left__ table is the table you include in the line of code first.

<div class="alert alert-warning">
    <b> The definitions presented here are the very basic definition of the join operations. There are extensions to these models that expand their functionalities.<b>
</div>

Let's take a look at several common join types.

---

### Left join

The left join is probably the most common join. It is called this way because the right table is added to the left table when the match is possible. In a sense, the left table is still intact inside the final resulting table. When you add columns to a table in a way all the original content of the original table is still present, we say the left table was __enriched__.

Records in an existing column that have no match are filled with a __null__.

<img src="../Figures/left_join.gif" width=400>

In [1]:
import dask.dataframe as dd
from dask import delayed
import pandas as pd

In [2]:
file_name_a = '../Files/entry_tableA.csv'

dda = dd.read_csv(file_name_a)

dda ## Just an instruction to read the contents of the file

,name,speed,flies,place
npartitions=1,,,,
,object,int64,bool,object
,...,...,...,...


In [3]:
type(dda)

dask.dataframe.core.DataFrame

In [4]:
dda.head() ## Finally reads file contents and displays

,name,speed,flies,place
0,Duck,6,True,pond
1,Goose,5,True,farm
2,Dog,4,False,farm
3,Wolf,5,False,woods


In [5]:
file_name_b = '../Files/entry_tableB.csv'

ddb = dd.read_csv(file_name_b)
ddb ## Just an instruction to read the contents of the file

,place,category
npartitions=1,,
,object,object
,...,...


In [6]:
ddb.head()

,place,category
0,farm,pet
1,woods,wild


In [7]:
type(ddb)

dask.dataframe.core.DataFrame

In [8]:
pdb

Automatic pdb calling has been turned ON


Finally, the join

In [9]:
dda.merge(ddb, on='place', how='left')

,name,speed,flies,place,category
npartitions=1,,,,,
,object,int64,bool,object,object
,...,...,...,...,...


In [10]:
result = dda.merge(ddb, on='place', how='left')

In [11]:
result.head() ## At this point, laziness gets executed

,name,speed,flies,place,category
0,Duck,6,True,pond,NaN
1,Goose,5,True,farm,pet
2,Dog,4,False,farm,pet
3,Wolf,5,False,woods,wild


We can also merge (or join) on more than one column. Let's exemplify with another dataframe where we store a "threat classification level".

In [12]:
ddc = dd.read_csv('../Files/entry_tableC.txt')

In [13]:
ddc.head()

,place,category,flies,threat_level
0,farm,pet,True,menace
1,woods,wild,False,dangerous


In [14]:
dda.head()

,name,speed,flies,place
0,Duck,6,True,pond
1,Goose,5,True,farm
2,Dog,4,False,farm
3,Wolf,5,False,woods


In [15]:
dda.merge(ddc, on=['place', 'flies'], how='left')

,name,speed,flies,place,category,threat_level
npartitions=1,,,,,,
,object,int64,bool,object,object,object
,...,...,...,...,...,...


In [16]:
res = dda.merge(ddc, on=['place', 'flies'], how='left')
res.head(10)

,name,speed,flies,place,category,threat_level
0,Duck,6,True,pond,NaN,NaN
1,Goose,5,True,farm,pet,menace
2,Dog,4,False,farm,NaN,NaN
3,Wolf,5,False,woods,wild,dangerous


---

### Right Join

The right join is the mirror logic of the left join. We now conserve the information of the entire right table inside the final table.

<img src="../Figures/right_join.gif" width=400>

In [17]:
dda.merge(ddb, on='place', how='left').head()

,name,speed,flies,place,category
0,Duck,6,True,pond,NaN
1,Goose,5,True,farm,pet
2,Dog,4,False,farm,pet
3,Wolf,5,False,woods,wild


In [18]:
dda.head()

,name,speed,flies,place
0,Duck,6,True,pond
1,Goose,5,True,farm
2,Dog,4,False,farm
3,Wolf,5,False,woods


In [19]:
ddb.head()

,place,category
0,farm,pet
1,woods,wild


In [20]:
dda.merge(ddb, on='place', how='right').head()

,name,speed,flies,place,category
0,Goose,5,True,farm,pet
1,Dog,4,False,farm,pet
2,Wolf,5,False,woods,wild


We now get a table where the only elements conserved from dataframe __A__ are the ones that match elements from the other dataframe. We will miss you, duck!

For comparison:

In [21]:
dda.merge(ddb, on='place', how='left').head()

,name,speed,flies,place,category
0,Duck,6,True,pond,NaN
1,Goose,5,True,farm,pet
2,Dog,4,False,farm,pet
3,Wolf,5,False,woods,wild


---

### Inner Join

An inner join conserves the elements from both dataframes that have a perfect match: 

<img src="../Figures/inner_join.gif" width=400>

Attention: we changed the example dataframes so it wold be more explicit.

In [22]:
ddd = dd.read_csv("../Files/entry_tableD.csv")
ddd.head()

,name,ability
0,Duck,floats
1,Goose,flies
2,Dog,runs
3,Wolf,snarls


In [23]:
dde = dd.read_csv("../Files/entry_tableE.csv")
dde.head()

,ability,object
0,floats,boat
1,snarls,wild
2,snores,human
3,sleeps,lazy


In [24]:
result = ddd.merge(dde, on='ability', how='inner') ## This is just a transformation. We haven't actually instructed any actions yet.

In [25]:
result.head()

,name,ability,object
0,Duck,floats,boat
1,Wolf,snarls,wild


---

### Full join (or outer merge)

As the name implies, a full join will result in a dataframe with all the data from the original dataframes. All rows are conserved, matches will be done when possible.

<img src="../Figures/full_join.gif" width=400>

In [26]:
result = ddd.merge(dde, on=['ability'], how='outer') ## This is just a transformation. We haven't actually instructed any actions yet.

In [27]:
result.head(10)

,name,ability,object
0,Duck,floats,boat
1,Goose,flies,NaN
2,Dog,runs,NaN
3,Wolf,snarls,wild
4,NaN,snores,human
5,NaN,sleeps,lazy


---

### Joins with duplicated lines

When you have duplicate lines in one or both dataframes you will also get then in the final dataframe, in every possible combination. For example, if you have 3 matches on the left dataframe and 2 matches on the right dataframe, you will have 3x2=6 records in the final dataframe.

<img src="../Figures/left_join_repetition.gif" width=400>

In [28]:
ddf = dd.read_csv("../Files/entry_tableF.csv")
ddf.head()

,name,attribute
0,Duck,fluffy
1,Duck,cute
2,Duck,cuddly
3,Goose,mean
4,Dog,cute


In [29]:
ddg = dd.read_csv("../Files/entry_tableG.csv")
ddg.head()

,name,ability
0,Duck,flies
1,Duck,swims
2,Goose,flies


In [30]:
result = ddf.merge(ddg, on=['name'], how='inner') ## This is just a transformation. We haven't actually instructed any actions yet.

In [31]:
result.head(100)

,name,attribute,ability
0,Duck,fluffy,flies
1,Duck,fluffy,swims
2,Duck,cute,flies
3,Duck,cute,swims
4,Duck,cuddly,flies
5,Duck,cuddly,swims
6,Goose,mean,flies


<div class="alert alert-warning">
    <b> There are many join logics. We just discussed the most frequently used.<b>
</div>

[If you use SQL or any type of relational database language, you will never know what joins may be necessary.](https://i.stack.imgur.com/ObDyr.png)

---

[Let's Continue in the next notebook](06.4-CommonOperationsII.ipynb)